# Entrenamiento de features por sesion

In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import datetime
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

Leo el set de datos y el de entrenamiento

In [2]:
data_raw = pd.read_csv('events_up_to_01062018.csv',low_memory=False)
data_val = pd.read_csv('labels_training_set.csv',low_memory=False)

In [3]:
df_labels = data_val.copy(deep=True) 
df = data_raw.copy(deep=True)

Leo los features obtenidos hasta el momento

In [4]:
sesiones = pd.read_csv('eventos_sesion.csv', low_memory = False)

In [5]:
features_jp = pd.read_csv('features_0.84927.csv', low_memory = False)

## XGboost entrenamiento 

### Por persona

In [10]:
df_train_por_persona = df_labels.merge(features_jp, left_on='person', right_on='person' , how='left')

In [11]:
df_train_por_persona.shape

(19414, 398)

In [12]:
df_train_por_persona=df_train_por_persona.fillna(0)

In [13]:
X_por_persona, y_por_persona = df_train_por_persona.iloc[:,2:],df_train_por_persona.iloc[:,1]
X_por_persona.head()

,busco_top_5_visitas,visito_mas_que_el_promedio,siempre_incrementando,inc_ultimo_mes,mismo_interes_ultimos_dos_meses,checkout_sum,checkout_mean,conversion_sum,conversion_mean,viewed_sum,...,device_type_Smartphone,device_type_Tablet,device_type_Unknown,channel_frecuente_Direct,channel_frecuente_Email,channel_frecuente_Organic,channel_frecuente_Paid,channel_frecuente_Referral,channel_frecuente_Social,channel_frecuente_Unknown
0,0,0,0,1,0,1.0,1.0,1.0,1.0,23.0,...,1,0,0,0,0,0,0,1,0,0
1,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,1,0,0,0
2,1,1,0,0,0,1.0,1.0,0.0,0.0,31.0,...,1,0,0,0,0,0,0,1,0,0
3,0,1,0,1,0,0.0,0.0,0.0,0.0,24.0,...,1,0,0,0,0,0,0,1,0,0
4,0,1,0,1,0,2.0,2.0,0.0,0.0,9.0,...,0,0,0,0,0,0,1,0,0,0


In [14]:
X_train_por_persona, X_test_por_persona, y_train_por_persona, y_test_por_persona = \
    train_test_split(X_por_persona, y_por_persona, test_size=0.2, random_state=123)

In [15]:
def xgb_classifier(X_train, X_test, y_train, y_test, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    alg = xgb.XGBClassifier(objective ='reg:linear', 
                colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 7,
                subsample = 0.9,
                gamma = 1,
                n_estimators = 50)
    
    print('\nXGBoost Classifier')
    if useTrainCV:
        print("Start Feeding Data")
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(X_train.values, label=y_train.values)
        # xgtest = xgb.DMatrix(X_test.values, label=y_test.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
                          early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    alg.fit(X_train, y_train, eval_metric='auc')
    
    pred_proba = alg.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, pred_proba)
    
    print('El puntaje auc es: {}'.format(auc))
    
    return alg

In [16]:
xgb_model_por_persona = xgb_classifier(X_train_por_persona, X_test_por_persona, \
                                       y_train_por_persona, y_test_por_persona,useTrainCV=True)


XGBoost Classifier
Start Feeding Data
El puntaje auc es: 0.869957967701


In [17]:
xgb_predict_por_persona = xgb_model_por_persona.predict_proba(X_test_por_persona)[:,1]

In [18]:
roc_auc_score(y_test_por_persona, xgb_predict_por_persona)

0.869957967701497

In [19]:
xgb_predict_por_persona.shape

(3883,)

### Por sesion


In [20]:
df_train_por_sesion = df_labels.merge(sesiones, left_on='person', right_on='person' , how='left')

In [21]:
df_train_por_sesion = df_train_por_sesion.fillna(0)

In [22]:
df_train_por_sesion.shape

(78882, 14)

In [23]:
X_por_sesion, y_por_sesion = df_train_por_sesion.iloc[:,2:],df_train_por_sesion.iloc[:,1]
X_por_sesion.head()

,Unnamed: 0,ad campaign hit,brand listing,visited site,searched products,staticpage,conversion,viewed product,lead,search engine hit,generic listing,checkout
0,84832.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,118033.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
3,122104.0,1.0,0.0,1.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
X_train_por_sesion, X_test_por_sesion, y_train_por_sesion, y_test_por_sesion = \
    train_test_split(X_por_sesion, y_por_sesion, test_size=0.2, random_state=123)

In [25]:
xgb_model_por_sesion = xgb_classifier(X_train_por_sesion, X_test_por_sesion,\
                                      y_train_por_sesion, y_test_por_sesion,useTrainCV=True)


XGBoost Classifier
Start Feeding Data
El puntaje auc es: 0.672044354562


In [26]:
xgb_predict_por_sesion = xgb_model_por_sesion.predict_proba(X_test_por_sesion)[:,1]

In [27]:
roc_auc_score(y_test_por_sesion, xgb_predict_por_sesion)

0.6720443545623446

In [28]:
xgb_predict_por_sesion.shape

(15777,)

Leo las personas a predecir

## Predicciones por sesion

In [29]:
personas_a_predecir = pd.read_csv('trocafone_kaggle_test.csv', low_memory=False)

In [30]:
personas_a_predecir_con_features = personas_a_predecir.merge(sesiones, on='person', how='left')

In [31]:
#personas_a_predecir_con_features = personas_a_predecir_con_features.merge(modelos, left_on='person', right_on='person' , how='left')

In [32]:
personas_a_predecir_con_features = personas_a_predecir_con_features.fillna(0)

In [33]:
personas_a_predecir_con_features_sin_person = personas_a_predecir_con_features.drop(columns = ['person'])

In [34]:
personas_a_predecir_con_features_sin_person.shape

(77336, 12)

In [35]:
xgb_predict_sesiones = xgb_model_por_sesion.predict_proba(personas_a_predecir_con_features_sin_person)[:,1]

In [36]:
xgb_predict_sesiones.shape

(77336,)

In [37]:
xgb_predict_por_persona = pd.DataFrame(xgb_predict_sesiones)

In [38]:
xgb_predict_por_persona.head()

,0
0,0.056551
1,0.049023
2,0.056551
3,0.085179
4,0.080756


In [39]:
xgb_predict_por_persona = xgb_predict_por_persona.merge(personas_a_predecir_con_features,left_index=True, right_index=True)

In [40]:
xgb_predict_por_persona=xgb_predict_por_persona[[0,"person"]]

In [41]:
xgb_predict_por_persona=xgb_predict_por_persona.groupby(["person"]).agg({0:'max'}).reset_index()

In [42]:
xgb_sesiones = xgb_predict_por_persona[[0]]

In [43]:
xgb_sesiones=pd.Series(xgb_sesiones[0])

In [44]:
xgb_sesiones.shape

(19415,)

## Predicciones por persona

In [45]:
personas_a_predecir_con_features = personas_a_predecir.merge(features_jp, on='person', how='left')

In [48]:
personas_a_predecir_con_features = personas_a_predecir_con_features.fillna(0)

In [49]:
personas_a_predecir_con_features = personas_a_predecir_con_features.drop(columns = ['person'])

In [50]:
xgb_persona = xgb_model_por_persona.predict_proba(personas_a_predecir_con_features)[:,1]

In [51]:
xgb_persona.shape

(19415,)

## Union

In [52]:
prediccion_final = 0.3*xgb_sesiones+0.7*xgb_persona

In [53]:
prediccion_final = pd.DataFrame(prediccion_final)

In [65]:
final_prediction_tp = pd.Series(0.3*xgb_sesiones+0.7*xgb_persona)

In [66]:
personas_a_predecir['label'] = final_prediction_tp

In [67]:
num = personas_a_predecir._get_numeric_data()
num[num < 0] = 0

In [68]:
personas_a_predecir.to_csv(path_or_buf = 'submit_kaggle.csv', index = False)

In [69]:
personas_a_predecir.head()

,person,label
0,4886f805,0.042685
1,0297fc1e,0.075151
2,2d681dd8,0.023351
3,cccea85e,0.098764
4,4c8a8b93,0.038066
